# 基于Embedding的RNN生成唐詩

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
import pandas as pd
import random
import re

In [2]:
special_words = ['<PAD>' , '<EOS>']

poem_word_count = {'五言絕句' : 24 , '五言律詩' : 48 , '七言絕句' : 32 , '七言律詩' : 64}

mode = '五言律詩' 
if mode == '五言絕句':
    file_name = 'dataset/{}.txt'.format('五言絕句')
elif mode == '五言律詩':
    file_name = 'dataset/{}.txt'.format('五言律詩')
elif mode == '七言絕句':
    file_name = 'dataset/{}.txt'.format('七言絕句')
elif mode == '七言律詩':
    file_name = 'dataset/{}.txt'.format('七言律詩')

# 讀取數據

In [3]:
# 詩集
count = 0
sources , targets , poems = [] , [] , []
f = open(file_name , 'r' , encoding = 'utf-8')
for i , line in enumerate(f.readlines()):
    try:
        title, content = line.strip().split(':')
        content = content.replace(' ' , '')
        content = content[:-1]
        content = content.replace('。' , '，') # 把'。'換成'，'應該會比較好訓練
        content += '。' 

        # 1.不符合字數 或 2.詞句中有不是中文&逗號&句號的都不考慮
        if len(content) == poem_word_count[mode] and re.search('[^\u4e00-\u9fa5。，]' , content) is None:
            source_temp = []
            for word in content:
                source_temp.append(word)
            source = source_temp + ['<EOS>']
            target = source[1:] + ['<PAD>']
            sources.append(source)
            targets.append(target)
            poems.append(source_temp)
        else:
            if i % 30 == 0:
                print('{} : {}'.format(mode , content)) # 顯示不列入訓練資料的詩句
                count += 1

    except ValueError:
        pass

五言律詩 : 獨羨爲僧者，塵緣不到門，采薪松落子，移徑竹生孫，古井汲雲氣，寒山招月魂，猶憐生死海，應□出前村。
五言律詩 : 霜壓危簷冷，蕭條秋滿林，青雲知己惠，黄菊故園心，百慮憑祛遣，頹齡仗制禁，緬思彭澤叟，履運□懷深。
五言律詩 : 壁立石青蒼，鄰溪復枕江，刊銘傳不朽，文筆老無雙，怪樹森圓幄，□□寄□幢，奇蹤□異景，□□□□□。
五言律詩 : 風掃柴門凈，銀燈吐玉花，納凉憑□石，待月上窗紗，老去心猶壯，秋來鬢任華，不妨高枕睡，無夢到公家。
五言律詩 : 駿骨直千金，難求起死針，雖云爲馬□，却似有人心，氈帳長城遠，旗亭委巷深，老身全頼此，完璧到如今。
五言律詩 : 雪後江南路，新晴野日濃，車輿通旅客，籬落聚村僮，澌解波□□，泥乾土未鬆，吾家有春酒，歸斸故園菘。
五言律詩 : 烏□□怪石，撐突倚山巔，□□群羊伏，昂昂萬馬聯，何年現蓬島，終古頌桑田，相揖閩王宅，東隅有九仙。
五言律詩 : 空庭風捲葉，壞廡雨淋碑，起廢須成厦，占孚預擊蓍，紅樓追曩集，碧落待誰詩，非有鈎牽□，□□亦未知。
五言律詩 : 道傍誰氏舘，□我坐開樽，翁嫗出迎客，兒童爲掃門，好花生竹所，流水浸雲根，儻遂卜隣約，爲農老此村。
五言律詩 : □客背時芳，何須晚節香，人間方夏熱，籬下自秋光，便是重陽日，真成六月霜，若教陶令見，應以醉爲鄉。
五言律詩 : 何地早芳菲？宛在長門殿，夭桃色若綬，穠李光如練，啼鳥弄花疎，遊蠭飲香徧，歎息春風起，飄零君不見。
五言律詩 : 源出崑崙中，長波接漢空，桃花來馬頰，竹箭入龍宮，德水千年變，榮光五色通，若披蘭葉檢，□□□□□。


# 數據預處理

In [4]:
# 統計每個字出現次數
all_words = []
for poem in poems:
    all_words += poem
    
# counter包含了每個字對應的頻率
counter = {}  
for word in all_words:
    if word not in counter.keys():
        counter[word] = 1
    else:
        counter[word] += 1
count_pairs = sorted(counter.items() , key = lambda x : x[1] , reverse = True) # 根據每個字出現的次數，由高排到低


# 取前多少個常用字，可以將len(words)換成其他數目，這裡是全部的字都收集起來
words = [i[0] for i in count_pairs]

words = special_words + words[:len(words) + 1]

In [5]:
# 每個字映射為一個數字ID
word_to_int = {}
for idx , word in enumerate(words):
    word_to_int[word] = idx
int_to_word = dict(zip(word_to_int.values() , word_to_int.keys()))

source_poem , target_poem = [] , []
for poem in sources:
    temp = []
    for word in poem:
        temp.append(word_to_int[word])
    source_poem.append(temp)
    
for poem in targets:
    temp = []
    for word in poem:
        temp.append(word_to_int[word])
    target_poem.append(temp)

# 構建網絡

In [6]:
# 超參數
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 64
# RNN Size(RNN隱層結點數量)
rnn_hidden_unit = 128
# Embedding Dimension Size
embedding_size = 200
# Sequence Length
seq_length = 20
# layer numbers(總共2層RNN)
num_layers = 2 
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 50
# vocabulary size
vocab_size = len(word_to_int)

In [7]:
# 輸入層
input_data = tf.placeholder(tf.int32, [None , None])
targets = tf.placeholder(tf.int32 , [None , None])
on_train = tf.placeholder(tf.bool , name = 'on_train') # 決定train模式或者test模式

# Word Embedding

In [8]:
# 進行embedding
# input_data: 輸入的tensor
# vocab_size: 詞匯表大小
# embedding_size: 嵌入維度
embedding = tf.Variable(tf.random_uniform([vocab_size, rnn_hidden_unit], -1 , 1))
inputs = tf.nn.embedding_lookup(embedding , input_data)

# Build RNN

In [9]:
def get_lstm_cell(rnn_hidden_unit):
    lstm_cell = tf.contrib.rnn.LSTMCell(rnn_hidden_unit)
    return lstm_cell

cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_hidden_unit) for _ in range(num_layers)])

'''
在測試時，batch_size為1(只有1筆要測試的資料)，因此分為兩種不同輸入的情況
在測試的時候必須要用到init_state_for_test，因為再生成第2個之後的詞，都必須要將生成的last_State餵到init_state_for_test
'''
# output => [batch_size , ? , rnn_size] = [64 , ? , 128]
init_state_for_train = cell.zero_state(batch_size , dtype = tf.float32)
def train():
    output , last_state = tf.nn.dynamic_rnn(cell , 
                                            inputs , 
                                            initial_state = init_state_for_train , 
                                            dtype = tf.float32)
    return tf.identity(output) , tf.identity(last_state)

init_state_for_test = cell.zero_state(1 , dtype = tf.float32)
def test():
    output , last_state = tf.nn.dynamic_rnn(cell , 
                                            inputs , 
                                            initial_state = init_state_for_test , 
                                            dtype = tf.float32)
    return tf.identity(output) , tf.identity(last_state)  

output , last_state = tf.cond(on_train , train , test)

# Build Neural Network(構建神經網絡，將RNN層與全連接層相連)

In [10]:
# [? , vocab_size]
output = tf.reshape(output , [-1 , rnn_hidden_unit])
weights = tf.Variable(tf.truncated_normal([rnn_hidden_unit , vocab_size]))
bias = tf.Variable(tf.zeros(shape = [vocab_size]))

logits = tf.nn.bias_add(tf.matmul(output , weights) , bias = bias)
# logits = tf.contrib.layers.fully_connected(output , vocab_size , activation_fn = None)

# 可以訓練的過程中看一下寫出什麼詩
predicting_logits = tf.nn.softmax(logits) 
predicting_logits_ = tf.argmax(predicting_logits , axis = 1)
predicting_logits_ = tf.reshape(predicting_logits_ , [batch_size , -1])

In [11]:
targets_ = tf.one_hot(tf.reshape(targets , [-1]) , depth = vocab_size)
loss = tf.nn.softmax_cross_entropy_with_logits(labels = targets_ , logits = logits)
total_loss = tf.reduce_mean(loss)

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
var_list = tf.trainable_variables()
gradients = tf.gradients(total_loss , var_list)
# 利用gradient clippling的方式來防止梯度爆炸
# 即通過設置一個threshold，當gradients超過這個threshold時，就將gradients變為threshold
# 這就保證了梯度不會變得很大
capped_gradients = [(tf.clip_by_value(grad, -1. , 1.)) for grad in gradients if grad is not None]      
train_op = optimizer.apply_gradients(zip(capped_gradients , var_list))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



# 構造batch

在機器人寫唐詩這兩個主題中，source與target分別如下:
    
    source :  得  不  到  的  就  更  加  愛  ,    太   容  易  來  的  就  不  理   睬    <EOS>
    target :  不  到  的  就  更  加  愛   ,  太   容   易  來  的  就  不  理  睬  <EOS>  <PAD>
     
其實target就是將source往右移動一格，達到錯位的效果

In [12]:
def get_batches(batch_i):    
    start_index = batch_i * batch_size
    end_index = start_index + batch_size
    source_batch = source_poem[start_index : end_index].copy()
    target_batch = target_poem[start_index : end_index].copy()
    
    # 找到這個batch的所有poem中最長的poem的長度  
    max_source_length = max([len(batch) for batch in source_batch]) 
    max_target_length = max([len(batch) for batch in target_batch])
    
    # 將batches中的每個詩句的長度都填充到與length一樣長，填充值為word_to_int['<PAD>']
    source_batch_pad , target_batch_pad = [] , []
    source_length , target_length = [] , []
    for i in range(0 , batch_size):
        source_length.append(len(source_batch[i]))
        target_length.append(len(target_batch[i]))
        
        source_temp = source_batch[i].copy()
        while len(source_temp) < max_source_length:
            source_temp.append(word_to_int[pad_token])
            
        target_temp = target_batch[i].copy()    
        while len(target_temp) < max_target_length:
            target_temp.append(word_to_int[pad_token])
            
        source_batch_pad.append(source_temp)
        target_batch_pad.append(target_temp)
        
    source_batch_pad = np.array(source_batch_pad)
    target_batch_pad = np.array(target_batch_pad)    
    source_length = np.array(source_length)
    target_length = np.array(target_length)        
    """
    source_batch_pad : [從 , 前 , 明 , 月 ,  光   ,  <EOS> ,  <PAD>   ,  <PAD>]
    target_batch_pad : [前 , 明 , 月 , 光 , <EOS> ,  <PAD> ,  <PAD>   ,  <PAD>]  
    source_batch_pad : [月 , 落 , 烏 , 啼 ,  霜   ,   滿    ,   天     ,  <EOS>]
    target_batch_pad : [落 , 烏 , 啼 , 霜 ,  滿   ,   天    ,  <EOS>   ,  <PAD>] 
    """    
    return source_batch_pad , target_batch_pad , source_length , target_length

# 模型訓練

In [13]:
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

n_chunk = len(source_poem) // batch_size
stop_early = 0
loss_ = []
for epoch_i in range(0 , num_epochs):
    loss_batch = []
    for batch_i in range(0 , n_chunk):
        source_batch_pad , target_batch_pad , _ , target_length = get_batches(batch_i)
        
        feed_dict = {input_data : source_batch_pad ,
                     targets : target_batch_pad ,  
                     on_train : True}
                             
        _ , loss = sess.run([train_op , total_loss] , feed_dict)
   
        # 每一個階段就來看一下結果
        if batch_i % show_every_n_batches == 0:
            index = random.randint(0 , batch_size - 1) # 隨機從0~batch_size選取一個index來觀看預測出來的詩句
            source_value, target_value , predicting_value = sess.run([input_data , targets , predicting_logits_ ] , feed_dict)
            
            source_value = source_value[index , :]
            target_value = target_value[index , :]
            predicting_value = predicting_value[index , :]
            
            source_result = ''
            for int_source in source_value:
                if int_to_word[int_source] == '<EOS>' : break
                source_result += int_to_word[int_source]
                
            target_result = ''
            for int_target in target_value:
                if int_to_word[int_target] == '<EOS>' : break
                target_result += int_to_word[int_target]
            
            predicting_result = ''
            for int_predicting in predicting_value:
                if int_to_word[int_predicting] == '<PAD>' : break
                predicting_result += int_to_word[int_predicting]

            print('=' * 90)    
            print('source_value : {}'.format(source_result))
            print('~' * 80)
            print('target_value : {}'.format(target_result))
            print('~' * 80)
            print('predicting_value : {}'.format(predicting_result))
            print('~' * 80)
            print('epoch_i : {}'.format(epoch_i))
            print('batch_i : {}'.format(batch_i))
            print('loss : {:.4f}\n'.format(loss))
            
        loss_batch.append(loss)
    
    loss_batch = np.array(loss_batch)  
    loss_mean = loss_batch.mean()  
    print('*' * 90)
    print('batch_loss_mean : {:.4f}'.format(loss_mean))
    print('stop_early : {}'.format(stop_early))
    
    # 隨著訓練的演進所存下的model會有不同的風格
    if epoch_i % 6 == 0 and epoch_i > 0:
        saver.save(sess , './model_{}/'.format(mode) , global_step = epoch_i)
    
    if epoch_i == 0:
        loss_.append(loss_mean)
    elif epoch_i > 0:    
        loss_ewm = pd.Series(loss_)
        loss_ewm = loss_ewm.ewm(alpha = 0.7 , adjust = False).mean()
        if loss_mean <= loss_ewm.iloc[-1]:
            print('New Record!\n')
        else: 
            print('No Improvement.\n')
            stop_early += 1
            if stop_early == 10: break 
        loss_ = list(loss_)  
        loss_.append(loss_mean)   

source_value : 十里尋幽寺，江山幾路分，逢人問息耗，指路上高雲，極浦明殘雨，寒流疊細紋，野鴉無意緒，嘹唳傍溪濆。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 里尋幽寺，江山幾路分，逢人問息耗，指路上高雲，極浦明殘雨，寒流疊細紋，野鴉無意緒，嘹唳傍溪濆。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 篛權汨汨汨，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
batch_i : 0
loss : 9.1148

source_value : 感夢通玄化，覃恩降紫宸，賜錢開漢府，分帛醉堯人，地隔朝宗慶，亭臨卜洛新，行看廣雲雨，二月次東巡。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 夢通玄化，覃恩降紫宸，賜錢開漢府，分帛醉堯人，地隔朝宗慶，亭臨卜洛新，行看廣雲雨，二月次東巡。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : ，，，，，風風風不，，風風不人，，風風不不，，風風不不，，風風不風，，風風風人，，風風風風，，<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
batch_i : 50
loss : 5.8347

source_value : 晚起家何事，無營地轉幽，竹光團野色，舍影漾江流，失

source_value : 撞玉鏘斷石，濺珠懸穉松，色資耳目凈，聲碎塵土空，靜窺發奇趣，飽飲生異容，詎知愚所樂，適與智者同。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 玉鏘斷石，濺珠懸穉松，色資耳目凈，聲碎塵土空，靜窺發奇趣，飽飲生異容，詎知愚所樂，適與智者同。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 雨清山，，風雲無翠城，山開三不遠，天火不埃中，山室山月節，天有不清鄉，不有無字樂，何有有人名。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
batch_i : 750
loss : 5.2737

source_value : 斷雲秋望闊，山靄夕成霞，凄冷初高月，喧闐未定鴉，伏羈良馬瘦，挂壁寶月斜，寂寂山城夜，明燈照鬢華。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 雲秋望闊，山靄夕成霞，凄冷初高月，喧闐未定鴉，伏羈良馬瘦，挂壁寶月斜，寂寂山城夜，明燈照鬢華。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 人隨水雨，山水落陽秋，水凉千寒樹，寒香月覺秋，山雲人自到，不樹上江明，不寂無中路，寒朝照月毛。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
batch_i : 800
loss : 4.7688

source_value : 地秘三茅勝，班高玉筍明，昔年同日喜，共

source_value : 虜寇日相持，如龍馬不肥，突圍金甲破，趂賊鐵槍飛，漢月堂堂上，胡雲慘慘微，黃河冰已合，猶未送征衣。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 寇日相持，如龍馬不肥，突圍金甲破，趂賊鐵槍飛，漢月堂堂上，胡雲慘慘微，黃河冰已合，猶未送征衣。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 水多無望，東何不自歸，不然千馬出，不氣玉衣旗，不府三堂秀，天爲不一然，何河何雪盡，何可見君鴻。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 1
batch_i : 550
loss : 4.6506

source_value : 欲扣隴頭信，侵晨踏雪來，不知今夜裏，先放幾花開，地有孤根在，春從何處回，向人如解語，約我辦樽罍。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 扣隴頭信，侵晨踏雪來，不知今夜裏，先放幾花開，地有孤根在，春從何處回，向人如解語，約我辦樽罍。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 知山頭路，相尋雲雪霜，山知天日雨，不見一年開，野僻山雲在，山風一處期，此來知此語，不我更詩中。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 1
batch_i : 600
loss : 4.3028

source_value : 庭菊種匪晚，秋風開爾遲，人嗟地力薄，我

source_value : 玉泉歸故剎，便老是僧期，亂木孤蟬後，寒山絕鳥時，若尋流水去，轉出白雲遲，見說千峰路，溪深復頂危。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 泉歸故剎，便老是僧期，亂木孤蟬後，寒山絕鳥時，若尋流水去，轉出白雲遲，見說千峰路，溪深復頂危。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 樹無不園，山覺老春房，野石隨燈裏，孤林落水稀，山知山水闊，不覺白雲深，莫說無家下，還流一幾頭。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 2
batch_i : 350
loss : 4.4847

source_value : 元日如常日，閒人更老人，後杯真壽酒，便服勝朝紳，何物稱三品，今朝過六旬，候門憐稚子，莫放杏梢春。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 日如常日，閒人更老人，後杯真壽酒，便服勝朝紳，何物稱三品，今朝過六旬，候門憐稚子，莫放杏梢春。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 日無何永，風居獨不成，風生無有酒，詩覺老詩衣，老日無無徑，三朝不一年，一來無不子，不遣白花紅。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 2
batch_i : 400
loss : 4.5321

source_value : 經過太怱怱，山行去路通，客衣沾曉露，縣

source_value : 曉氣生綠水，春條露霏霏，林間棲鳥散，遠念征人起，幽花宿含彩，早蝶寒弄翅，君行非晨風，詎能從門至。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 氣生綠水，春條露霏霏，林間棲鳥散，遠念征人起，幽花宿含彩，早蝶寒弄翅，君行非晨風，詎能從門至。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 日連雲水，風風一氣霏，風間有鳥道，雲岫一帆過，山人落雲月，山晚不聲花，不看不所興，何知知此路。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 3
batch_i : 150
loss : 4.4064

source_value : 少年負志氣，信道不從時，只言繩自直，安知室可欺，百勝難慮敵，三折乃良醫，人生不失意，焉能慕知己。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 年負志氣，信道不從時，只言繩自直，安知室可欺，百勝難慮敵，三折乃良醫，人生不失意，焉能慕知己。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 日無一業，一有不爲容，一是無不在，不得不相人，一年無得薄，一伏不如工，不物無可意，何能爲吾音。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 3
batch_i : 200
loss : 4.6281

source_value : 遠水沈西日，寒沙聚夜鷗，平湖乘月滿，飛

...

source_value : 毀以是爲非，譽以非爲是，是非自有真，不在譽與毀，譽堯復毀桀，兩忘固佳耳，不見繒布紋，眼醉眩成綺。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 以是爲非，譽以非爲是，是非自有真，不在譽與毀，譽堯復毀桀，兩忘固佳耳，不見繒布紋，眼醉眩成綺。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 譽不吾己，吾以不所人，一非雖可命，不復一不噉，何以一何譽，不子不有趣，不知一繳姿，不前不雲縷。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 29
batch_i : 450
loss : 4.1480

source_value : 風邊藥氣濃，烟畔柳枝弱，惜許兩春事，潘郎行色作，維南有岑嶺，底處端州郭，客子何當歸，莫待蕉花落。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 邊藥氣濃，烟畔柳枝弱，惜許兩春事，潘郎行色作，維南有岑嶺，底處端州郭，客子何當歸，莫待蕉花落。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 雨不草薄，風雨春已青，春此不枝雨，不輿一晼難，風舟有餘士，不事不不上，何裏不所爲，何恨花葉醉。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 29
batch_i : 500
loss : 4.0615

source_value : 花塹接滄洲，陰雲閑楚丘，雨聲雖到夜

source_value : 令弟與名兄，高才振兩京，別序聞鴻燕，離章動鶺鴒，遠朋馳翰墨，勝地寫丹青，風月相思夜，勞望潁川星。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 弟與名兄，高才振兩京，別序聞鴻燕，離章動鶺鴒，遠朋馳翰墨，勝地寫丹青，風月相思夜，勞望潁川星。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 尹今君重，相名不振朝，一來不風鵠，歸弦入玉鴒，風山無道墨，萬地入金青，不雨今思處，何歌不川濆。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 30
batch_i : 250
loss : 4.2214

source_value : 樽前逢臘近，行李獨西征，事歷風霜慣，春回雨露生，一經家自學，千佛世題名，昨夜瞻奎宿，寒芒萬丈明。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 前逢臘近，行李獨西征，事歷風霜慣，春回雨露生，一經家自學，千佛世題名，昨夜瞻奎宿，寒芒萬丈明。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 酒一酒酒，春路幾相東，不事三流裏，心歸草露深，不樽無有在，一古月無名，不夜無南上，相燈不里深。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 30
batch_i : 300
loss : 4.0415

source_value : 柳新春水湄，春岸草離離，祖席觴云盡

source_value : 山峨來夾案，泉急自穿渠，密記東陽訣，高題上帝居，牙兒藏寶篋，山子走籃輿，曲折璣衡轉，溪坳可結廬。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 峨來夾案，泉急自穿渠，密記東陽訣，高題上帝居，牙兒藏寶篋，山子走籃輿，曲折璣衡轉，溪坳可結廬。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 林千不岸，雲落不潺雲，野雨無山路，高談舊古書，山籤知舊釡，花水有青輿，不水青衡級，何山有駐廬。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 31
batch_i : 0
loss : 3.8201

source_value : 茂德來徵應，流泉入詠歌，含靈符上善，作字表中和，有草恆垂露，無風欲偃波，爲看人共水，清白定誰多。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 德來徵應，流泉入詠歌，含靈符上善，作字表中和，有草恆垂露，無風欲偃波，爲看人共水，清白定誰多。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 苑不何詔，高年洽太中，一情不萬國，出媲出神猷，天子無何舌，無人不落松，不君一世笑，不絕自何知。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 31
batch_i : 50
loss : 4.0260

source_value : 白日移歌袖，清霄近笛牀，翠眉縈度曲，雲鬢

source_value : 强把殘編閲，支頤坐北堂，味從中夜永，痛偶暫時忘，意外逢奇藥，人間欠此方，倉公如欲覓，寄語不珍藏。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 把殘編閲，支頤坐北堂，味從中夜永，痛偶暫時忘，意外逢奇藥，人間欠此方，倉公如欲覓，寄語不珍藏。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 飲陶書客，詩頤不舊風，一深一酒話，心覺不相閑，不氣無春偉，人無有一心，何皇不有問，不語不勝禽。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 31
batch_i : 750
loss : 4.4506

source_value : 地平彌望水，舟穩滿帆風，黄葉新霜後，青山落照中，封疆天象別，風俗島夷同，黯黯危橋夜，疏燈照閉蓬。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 平彌望水，舟穩滿帆風，黄葉新霜後，青山落照中，封疆天象別，風俗島夷同，黯黯危橋夜，疏燈照閉蓬。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 僻無萬地，天走不平檣，山葉飛秋意，青山暮日中，風疆隨下遠，歸雨水邊秋，此黯東亭上，何鐘照夜門。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 31
batch_i : 800
loss : 4.0471

source_value : 世事關分合，人情看短長，君應愁富貴

source_value : 塢中元巳日，小雨浥芳菲，祓禊歲時共，招尋吾土非，竹光迷野徑，花氣襲人衣，頼有盈川遇，還同池上歸。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 中元巳日，小雨浥芳菲，祓禊歲時共，招尋吾土非，竹光迷野徑，花氣襲人衣，頼有盈川遇，還同池上歸。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 上有自日，天雨不餘草，不禊無寒時，不邀一道人，風風明遠色，雲影滿人衣，不有山樽下，何家一上山。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 32
batch_i : 500
loss : 4.2967

source_value : 不是將軍勇，胡兵豈易當，雨曾淋火陣，箭又中金瘡，鐵嶺全無土，豺羣亦有狼，因思無戰日，天子是陶唐。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 是將軍勇，胡兵豈易當，雨曾淋火陣，箭又中金瘡，鐵嶺全無土，豺羣亦有狼，因思無戰日，天子是陶唐。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 見山何客，何爲不易知，一餘收浪月，天劃上天城，天馬無無敵，金狼不自期，何君一限處，不地有東潛。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 32
batch_i : 550
loss : 4.3126

source_value : 往年珠履客，獨有老夫存，觸目景非昔

In [32]:
def compose_poem(start_word):
    # 定義啟動的單詞
    begin_word = [word_to_int[start_word]]
    begin_input = np.full([1 , len(begin_word)] , begin_word)
    gen_sentences = [start_word]
    word = ''
    
    time_step = 0
    while word != '<EOS>':
        state_current = sess.run(init_state_for_test)
    
        feed_dict = {input_data : begin_input ,
                     on_train : False,
                     init_state_for_test : state_current}
        
        prob = sess.run(tf.reshape(predicting_logits , [1 , time_step + 1 , vocab_size]) , feed_dict)
        time_step += 1
        prob = prob[: , -1 , :][0]

  
        # 以所有詞的機率分布選出下一個詞，在這裡先以機率最大的那個詞先當作下一個詞
        # 但也可以裡用這個機率分布sample出下一個詞，這樣每次產生的詩會比較多元
        int_ = prob.argmax()
        word = int_to_word[int_]
    
        begin_word.append(word_to_int[word])
        begin_input = np.full([1 , len(begin_word)] , begin_word)
        gen_sentences.append(word)
     
    print('=' * 90)
    print(''.join(gen_sentences))

compose_poem('花')
compose_poem('天')
compose_poem('狂')
compose_poem('骨')   

花落青山裏，雲端白鳥飛，山川無限日，山色半生關，野寺逢僧舍，山房見客心，何當一樽酒，不待一枝開。<EOS>
天下一何處，人間一笑中，一身猶自苦，萬物自相親，不見山中老，何妨世事疏，不知人事少，不復問詩書。<EOS>
狂氣不可見，一笑不能寐，一笑無所有，一笑不可量，一笑不可知，一笑不可知，不知一一字，不復問何時。<EOS>
骨肉不可得，人情不可量，一身猶自苦，萬物自相忘，不見三年別，何妨一笑長，不知天下士，不復問人情。<EOS>


In [34]:
def compose_poem_(start_word):
    # 定義啟動的單詞
    begin_word = [word_to_int[start_word]]
    begin_input = np.full([1 , len(begin_word)] , begin_word)
    gen_sentences = [start_word]
    word = ''
    
    time_step = 0
    while word != '<EOS>':
        state_current = sess.run(init_state_for_test)
    
        feed_dict = {input_data : begin_input ,
                     on_train : False,
                     init_state_for_test : state_current}
        
        prob = sess.run(tf.reshape(predicting_logits , [1 , time_step + 1 , vocab_size]) , feed_dict)
        time_step += 1
        prob = prob[: , -1 , :][0]

  
        # 以所有詞的機率分布sample出下一個詞，但不是挑出機率最大的那個詞當作下一個詞
        int_ = np.random.choice(len(prob) , 1 , p = prob)
        word = int_to_word[int_[0]]
    
        begin_word.append(word_to_int[word])
        begin_input = np.full([1 , len(begin_word)] , begin_word)
        gen_sentences.append(word)
     
    print('=' * 90)
    print(''.join(gen_sentences))

compose_poem_('鏡')
compose_poem_('花')
compose_poem_('水')
compose_poem_('月')  

鏡中多萬里，句事正精明，河舫槐花好，池臺景象歸，櫂朝無片入，人世兩南飛，溪口收泥潦，田園盛美氓。<EOS>
花發人爭小，人忙歲已暮，人不嘆失饑，行取方金軫，盤村起蟬茸，碎葉侵桑絮，平生見狐女，柳林相問寧。<EOS>
水流峨竹凉，江流多壯心，逸興常滿眼，朝騎指光陰，念欲此同來，風雷方眼明，下頭晉江蘺，怒逐湘川去。<EOS>
月皎衆園雨，山連杜陵寒，杏園菊簟散，草木鳥聲同，空恨人間事，無時墓守詩，有篇聊飲酒，終將莫嘲欣。<EOS>
